# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-5f586d67c0-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series ("Association Football Players explorative search") 

Consider the following exploratory information need:

> investigate the association football players and find the main BGPs related to this sport. Compare players per countries, awards of the players and participation in the FIFA World Cup.

**Important consideration: in wikidata, we are talking of ASSOCIATION FOOTBALL**

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q11571`| Cristiano Ronaldo       | node |





Also consider

```
wd:Q11571 ?p ?obj .
```

is the BGP to retrieve all **properties of Cristiano Ronaldo**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for association football player

2. Identify the BGP for association football club

3. Identify the BGP for association football league

4. Return for each european country the number of association football leagues (the result set must be a list of triples country IRI, label and #leagues).

5. Identify the BGP for the ballon d'Or award

6. Return the countries which have at least two different players of their nationality (consider the country of citizenship) who won the ballon d'Or (the result set must be a list of triples country IRI, label and #different players).

7. Identify the BGP of the FIFA World Cup

8. Who are the players who partecipated in more than 4 world's cup editions? (the result set must be a list of triples player IRI, label and #participations).

9. Consider the countries that won the FIFA World Cup at least twice. Find the players of those countries won the ballon d'Or (the result set must be a list of couples player IRI and label).


## Task 1
Identify the BGP for association football player

In [141]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   
   wd:Q11571 ?p ?obj .
   
   ?p sc:name ?name.
}
order by ?name
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3427'), ('name', 'AS.com athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6188'), ('name', 'BDFA player ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3655'), ('name', 'BDFutbol player ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5361'), ('name', 'BNB person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3630'), ('name', 'Babelio author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P950'), ('name', 'Biblioteca Nacional de España ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5019'), ('name', 'Brockhaus Enzyklopädie online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9368'), ('name', 'CNA topic ID')]
[('p', 'http://www.wikidata.org/prop/dir

In [3]:
# Let's look at the occupation of Ronaldo
queryString = """
SELECT DISTINCT ?occupation ?name
WHERE
{
    wd:Q11571 wdt:P106 ?occupation.
    ?occupation sc:name ?name
}
"""

print("Results")
x = run_query(queryString)

Results
[('occupation', 'http://www.wikidata.org/entity/Q937857'), ('name', 'association football player')]
1


In [4]:
# getting all players with occupation = association football player
queryString = """
SELECT DISTINCT ?afp ?name
WHERE
{
    ?afp wdt:P106 wd:Q937857.
    ?afp sc:name ?name
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('afp', 'http://www.wikidata.org/entity/Q259758'), ('name', 'Andrés Mendoza')]
[('afp', 'http://www.wikidata.org/entity/Q8135287'), ('name', 'Naoki Kurimoto')]
[('afp', 'http://www.wikidata.org/entity/Q8157342'), ('name', 'Lamberto Anichini')]
[('afp', 'http://www.wikidata.org/entity/Q8196829'), ('name', 'Alvin Avinesh')]
[('afp', 'http://www.wikidata.org/entity/Q8199060'), ('name', 'André Advíncula')]
[('afp', 'http://www.wikidata.org/entity/Q8199367'), ('name', 'Andrés Mendoza')]
[('afp', 'http://www.wikidata.org/entity/Q8201109'), ('name', 'Antonino Gart')]
[('afp', 'http://www.wikidata.org/entity/Q8204020'), ('name', 'Argemiro Veiga Gonçalves')]
[('afp', 'http://www.wikidata.org/entity/Q8238626'), ('name', 'Jiří Adamec')]
[('afp', 'http://www.wikidata.org/entity/Q29162'), ('name', 'Tomáš Rosický')]
[('afp', 'http://www.wikidata.org/entity/Q3027395'), ('name', 'Diego Héctor Garay')]
[('afp', 'http://www.wikidata.org/entity/Q8351764'), ('name', 'Cristóbal Martí')]
[('afp', '

Final query for this task

In [5]:
# write your final query
# getting all players with the same occupation as Ronaldo
queryString = """
SELECT DISTINCT ?afp ?name
WHERE
{
    wd:Q11571 wdt:P106 ?occupation.
    ?afp wdt:P106 ?occupation.
    ?afp sc:name ?name
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('afp', 'http://www.wikidata.org/entity/Q259758'), ('name', 'Andrés Mendoza')]
[('afp', 'http://www.wikidata.org/entity/Q8135287'), ('name', 'Naoki Kurimoto')]
[('afp', 'http://www.wikidata.org/entity/Q8157342'), ('name', 'Lamberto Anichini')]
[('afp', 'http://www.wikidata.org/entity/Q8196829'), ('name', 'Alvin Avinesh')]
[('afp', 'http://www.wikidata.org/entity/Q8199060'), ('name', 'André Advíncula')]
[('afp', 'http://www.wikidata.org/entity/Q8199367'), ('name', 'Andrés Mendoza')]
[('afp', 'http://www.wikidata.org/entity/Q8201109'), ('name', 'Antonino Gart')]
[('afp', 'http://www.wikidata.org/entity/Q8204020'), ('name', 'Argemiro Veiga Gonçalves')]
[('afp', 'http://www.wikidata.org/entity/Q8238626'), ('name', 'Jiří Adamec')]
[('afp', 'http://www.wikidata.org/entity/Q29162'), ('name', 'Tomáš Rosický')]
[('afp', 'http://www.wikidata.org/entity/Q3027395'), ('name', 'Diego Héctor Garay')]
[('afp', 'http://www.wikidata.org/entity/Q8351764'), ('name', 'Cristóbal Martí')]
[('afp', '

## Task 2
Identify the BGP for association football club

In [6]:
# write your queries

In [7]:
# getting all objects related to association football player
queryString = """
SELECT DISTINCT  ?o ?name
WHERE
{
    wd:Q937857 ?p ?o .
    ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/P2369'), ('name', 'Soccerway player ID')]
[('o', 'http://www.wikidata.org/entity/P3908'), ('name', 'Reprezentacija player ID')]
[('o', 'http://www.wikidata.org/entity/Q18536342'), ('name', 'competitive player')]
[('o', 'http://www.wikidata.org/entity/P2446'), ('name', 'Transfermarkt player ID')]
[('o', 'http://www.wikidata.org/entity/Q7217464'), ('name', 'Category:Association football players')]
[('o', 'http://www.wikidata.org/entity/P2823'), ('name', 'Royal Belgian Football Association player ID')]
[('o', 'http://www.wikidata.org/entity/Q15944511'), ('name', 'association football team')]
[('o', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
[('o', 'http://www.wikidata.org/entity/Q28640'), ('name', 'profession')]
[('o', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
[('o', 'http://www.wikidata.org/entity/Q31930761'), ('name', "men's association football")]
[('o', 'http://www.wi

In [8]:
# we know that association football club is wd:Q476028
# and predicate instance of wdt:P31
# now get all instances of association football club entity
queryString = """
SELECT DISTINCT ?instance ?name
WHERE
{
    ?instance wdt:P31 wd:Q476028.
    ?instance sc:name ?name.
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('instance', 'http://www.wikidata.org/entity/Q4647107'), ('name', 'A.D. Dili Leste')]
[('instance', 'http://www.wikidata.org/entity/Q8186475'), ('name', 'Tahuichi Academy')]
[('instance', 'http://www.wikidata.org/entity/Q8208548'), ('name', 'Atlético Trujillo')]
[('instance', 'http://www.wikidata.org/entity/Q2631783'), ('name', 'Cizrespor')]
[('instance', 'http://www.wikidata.org/entity/Q3448775'), ('name', 'FK Sloboda Mrkonjić Grad')]
[('instance', 'http://www.wikidata.org/entity/Q5100109'), ('name', 'Chinandega FC')]
[('instance', 'http://www.wikidata.org/entity/Q5379014'), ('name', 'Enkaspor')]
[('instance', 'http://www.wikidata.org/entity/Q7280157'), ('name', "Radford Highlanders men's soccer")]
[('instance', 'http://www.wikidata.org/entity/Q7300846'), ('name', 'Real Arroyo Seco')]
[('instance', 'http://www.wikidata.org/entity/Q14252413'), ('name', 'Rapid JC')]
[('instance', 'http://www.wikidata.org/entity/Q14919844'), ('name', "Fortuna '54")]
[('instance', 'http://www.wik

Final query for this task

In [9]:
# we know that association football club is wd:Q476028
# and predicate instance of wdt:P31
# now get all instances of association football club entity
queryString = """
SELECT DISTINCT ?instance ?name
WHERE
{
    ?instance wdt:P31 wd:Q476028.
    ?instance sc:name ?name.
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('instance', 'http://www.wikidata.org/entity/Q4647107'), ('name', 'A.D. Dili Leste')]
[('instance', 'http://www.wikidata.org/entity/Q8186475'), ('name', 'Tahuichi Academy')]
[('instance', 'http://www.wikidata.org/entity/Q8208548'), ('name', 'Atlético Trujillo')]
[('instance', 'http://www.wikidata.org/entity/Q2631783'), ('name', 'Cizrespor')]
[('instance', 'http://www.wikidata.org/entity/Q3448775'), ('name', 'FK Sloboda Mrkonjić Grad')]
[('instance', 'http://www.wikidata.org/entity/Q5100109'), ('name', 'Chinandega FC')]
[('instance', 'http://www.wikidata.org/entity/Q5379014'), ('name', 'Enkaspor')]
[('instance', 'http://www.wikidata.org/entity/Q7280157'), ('name', "Radford Highlanders men's soccer")]
[('instance', 'http://www.wikidata.org/entity/Q7300846'), ('name', 'Real Arroyo Seco')]
[('instance', 'http://www.wikidata.org/entity/Q14252413'), ('name', 'Rapid JC')]
[('instance', 'http://www.wikidata.org/entity/Q14919844'), ('name', "Fortuna '54")]
[('instance', 'http://www.wik

## Task 3
Identify the BGP for association football league

In [10]:
# write your queries

In [11]:
# we know that league predicate is wdt:P118(from task 1)
# lets get ronaldo's league
queryString = """
SELECT DISTINCT  ?p ?name
WHERE
{
    wd:Q11571 wdt:P118 ?p.
    ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q182994'), ('name', 'Liga Portugal')]
[('p', 'http://www.wikidata.org/entity/Q15804'), ('name', 'Serie A')]
[('p', 'http://www.wikidata.org/entity/Q324867'), ('name', 'La Liga')]
[('p', 'http://www.wikidata.org/entity/Q9448'), ('name', 'Premier League')]
4


In [12]:
# get class of the instance Premier League(wd:Q9448)
queryString = """
SELECT DISTINCT  ?p ?name
WHERE
{
    wd:Q9448 wdt:P31 ?p.
    ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q15991290'), ('name', 'professional sports league')]
[('p', 'http://www.wikidata.org/entity/Q15991303'), ('name', 'association football league')]
2


In [13]:
# now we know association football league is wd:Q15991303
queryString = """
SELECT DISTINCT  ?p ?name
WHERE
{
    ?p wdt:P31 wd:Q15991303.
    ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)


Results
[('p', 'http://www.wikidata.org/entity/Q304228'), ('name', 'Uzbekistan Pro League')]
[('p', 'http://www.wikidata.org/entity/Q2950655'), ('name', 'Ligue 2')]
[('p', 'http://www.wikidata.org/entity/Q2951924'), ('name', "Moldovan women's football championship")]
[('p', 'http://www.wikidata.org/entity/Q973312'), ('name', 'Championnat National 3')]
[('p', 'http://www.wikidata.org/entity/Q2792041'), ('name', '3. divisjon')]
[('p', 'http://www.wikidata.org/entity/Q1162407'), ('name', 'Nemzeti Bajnokság III')]
[('p', 'http://www.wikidata.org/entity/Q1476473'), ('name', 'Landesliga Niederrhein')]
[('p', 'http://www.wikidata.org/entity/Q162533'), ('name', 'Russian Football National League Competition')]
[('p', 'http://www.wikidata.org/entity/Q1707697'), ('name', 'Liga III')]
[('p', 'http://www.wikidata.org/entity/Q219586'), ('name', 'A-League')]
[('p', 'http://www.wikidata.org/entity/Q238113'), ('name', 'Slovak Super Liga')]
[('p', 'http://www.wikidata.org/entity/Q475887'), ('name', 'Bre

Final query for this task

In [52]:
# now we know association football league is wd:Q15991303
queryString = """
SELECT DISTINCT  ?p ?name
WHERE
{
    wd:Q15991303 ?p ?obj.
    ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
9


## Task 4
Return for each european country the number of association football leagues (the result set must be a list of triples country IRI, label and #leagues).

In [15]:
# write your queries

In [56]:
# find property of afl that shows country

#  wd:Q15991303 = association football league
# wdt:P31 = instance of

queryString = """
SELECT DISTINCT  ?p ?obj ?pName ?objName
WHERE
{
    ?randomAFL wdt:P31 wd:Q15991303 .
    ?randomAFL ?p ?obj .
    
    ?p sc:name ?pName .
    ?obj sc:name ?objName .
    
    Filter regex(?objName, "Italy") .
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('pName', 'country'), ('objName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('obj', 'http://www.wikidata.org/entity/Q172579'), ('pName', 'country'), ('objName', 'Kingdom of Italy')]
2


In [59]:
# find property of Italy that shows it is in europe

#  wd:Q38 = Italy

queryString = """
SELECT DISTINCT  ?p ?obj ?pName ?objName
WHERE
{
    wd:Q38 ?p ?obj .
    
    ?p sc:name ?pName .
    ?obj sc:name ?objName .
    
    Filter regex(?objName, "Europe") .
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('obj', 'http://www.wikidata.org/entity/Q46'), ('pName', 'continent'), ('objName', 'Europe')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('obj', 'http://www.wikidata.org/entity/Q46'), ('pName', 'part of'), ('objName', 'Europe')]
[('p', 'http://www.wikidata.org/prop/direct/P421'), ('obj', 'http://www.wikidata.org/entity/Q5412122'), ('pName', 'located in time zone'), ('objName', 'Europe/Rome')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('obj', 'http://www.wikidata.org/entity/Q151991'), ('pName', 'member of'), ('objName', 'European Southern Observatory')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('obj', 'http://www.wikidata.org/entity/Q42262'), ('pName', 'member of'), ('objName', 'European Space Agency')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('obj', 'http://www.wikidata.org/entity/Q81299'), ('pName', 'member of'), ('objName', 'Organization for Security and Co-operation in Europe')]
[('p

In [63]:
# use property country in afl to group by counties
# use property continent in country to filter European counties

#  wd:Q15991303 = association football league
#  wdt:P31 = instance of

# P17 = country
# P30 = continent
# Q46 = Europe

queryString = """
SELECT DISTINCT  ?country ?countryName (count(?afl) as ?numberofLeagues)
WHERE
{
    ?afl wdt:P31 wd:Q15991303 .
    ?afl wdt:P17 ?country .
    ?country wdt:P30 wd:Q46 .
    
    ?country sc:name ?countryName .
}
group by ?country ?countryName
order by desc(?numberofLeagues)
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q213'), ('countryName', 'Czech Republic'), ('numberofLeagues', '199')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('numberofLeagues', '130')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('numberofLeagues', '91')]
[('country', 'http://www.wikidata.org/entity/Q20'), ('countryName', 'Norway'), ('numberofLeagues', '72')]
[('country', 'http://www.wikidata.org/entity/Q218'), ('countryName', 'Romania'), ('numberofLeagues', '62')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('numberofLeagues', '32')]
[('country', 'http://www.wikidata.org/entity/Q55'), ('countryName', 'Netherlands'), ('numberofLeagues', '22')]
[('country', 'http://www.wikidata.org/entity/Q713750'), ('countryName', 'West Germany'), ('numberofLeagues', '18')]
[('country', 'http://www.wikidata.org/entity/Q1206012'), ('countryName', 'German Reich'), ('numberofLeague

Final query for this task

In [31]:
# write your final query

In [64]:
# use property country in afl to group by counties
# use property continent in country to filter European counties

#  wd:Q15991303 = association football league
#  wdt:P31 = instance of

# P17 = country
# P30 = continent
# Q46 = Europe

queryString = """
SELECT DISTINCT  ?country ?countryName (count(?afl) as ?numberofLeagues)
WHERE
{
    ?afl wdt:P31 wd:Q15991303 .
    ?afl wdt:P17 ?country .
    ?country wdt:P30 wd:Q46 .
    
    ?country sc:name ?countryName .
}
group by ?country ?countryName
order by desc(?numberofLeagues)
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q213'), ('countryName', 'Czech Republic'), ('numberofLeagues', '199')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('numberofLeagues', '130')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('numberofLeagues', '91')]
[('country', 'http://www.wikidata.org/entity/Q20'), ('countryName', 'Norway'), ('numberofLeagues', '72')]
[('country', 'http://www.wikidata.org/entity/Q218'), ('countryName', 'Romania'), ('numberofLeagues', '62')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('numberofLeagues', '32')]
[('country', 'http://www.wikidata.org/entity/Q55'), ('countryName', 'Netherlands'), ('numberofLeagues', '22')]
[('country', 'http://www.wikidata.org/entity/Q713750'), ('countryName', 'West Germany'), ('numberofLeagues', '18')]
[('country', 'http://www.wikidata.org/entity/Q1206012'), ('countryName', 'German Reich'), ('numberofLeague

## Task 5
Identify the BGP for the ballon d'Or award

In [32]:
# write your queries

In [33]:
# awards receied by ronaldo
queryString = """
SELECT DISTINCT  ?p ?name
WHERE
{
    wd:Q11571 wdt:P166 ?p.
    ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1750244'), ('name', 'FWA Footballer of the Year')]
[('p', 'http://www.wikidata.org/entity/Q10726649'), ('name', "PFA Fans' Player of the Year")]
[('p', 'http://www.wikidata.org/entity/Q20888805'), ('name', 'Commander of the Order of Merit of Portugal')]
[('p', 'http://www.wikidata.org/entity/Q1146578'), ('name', 'UEFA Team of the Year')]
[('p', 'http://www.wikidata.org/entity/Q182529'), ('name', 'FIFA World Player of the Year')]
[('p', 'http://www.wikidata.org/entity/Q260117'), ('name', "UEFA Men's Player of the Year Award")]
[('p', 'http://www.wikidata.org/entity/Q166177'), ('name', "Ballon d'Or")]
[('p', 'http://www.wikidata.org/entity/Q20860117'), ('name', 'Officer of the Order of Prince Henry')]
[('p', 'http://www.wikidata.org/entity/Q20859673'), ('name', 'Grand Officer of the Order of Prince Henry')]
[('p', 'http://www.wikidata.org/entity/Q2986505'), ('name', 'Sir Matt Busby Player of the Year')]
[('p', 'http://www.wikidata.org/entit

In [34]:
# people who recieved wdt:P166 
# FIFABallon d'Or 
queryString = """
SELECT DISTINCT  ?p ?name
WHERE
{
    ?p wdt:P166 wd:Q166177.
    ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q162043'), ('name', 'Eusébio')]
[('p', 'http://www.wikidata.org/entity/Q128829'), ('name', 'Michael Owen')]
[('p', 'http://www.wikidata.org/entity/Q102027'), ('name', 'Fabio Cannavaro')]
[('p', 'http://www.wikidata.org/entity/Q36386'), ('name', 'Lothar Matthäus')]
[('p', 'http://www.wikidata.org/entity/Q178628'), ('name', 'Paolo Rossi')]
[('p', 'http://www.wikidata.org/entity/Q216140'), ('name', 'Stanley Matthews')]
[('p', 'http://www.wikidata.org/entity/Q310657'), ('name', 'Matías Fernandez')]
[('p', 'http://www.wikidata.org/entity/Q483145'), ('name', 'Luís Figo')]
[('p', 'http://www.wikidata.org/entity/Q483577'), ('name', 'Rivaldo')]
[('p', 'http://www.wikidata.org/entity/Q483837'), ('name', 'Luka Modrić')]
[('p', 'http://www.wikidata.org/entity/Q39444'), ('name', 'Ronaldinho')]
[('p', 'http://www.wikidata.org/entity/Q483629'), ('name', 'Marco van Basten')]
[('p', 'http://www.wikidata.org/entity/Q17163'), ('name', 'Johan Cruyff')]
[('p',

Final query for this task

In [68]:
# write your final query

# wdt:P166 = award received  
# wd:Q166177 = Ballon d'Or 

queryString = """
SELECT DISTINCT  ?p ?name
WHERE
{
    ?p wdt:P166 wd:Q166177.
    ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q162043'), ('name', 'Eusébio')]
[('p', 'http://www.wikidata.org/entity/Q128829'), ('name', 'Michael Owen')]
[('p', 'http://www.wikidata.org/entity/Q102027'), ('name', 'Fabio Cannavaro')]
[('p', 'http://www.wikidata.org/entity/Q36386'), ('name', 'Lothar Matthäus')]
[('p', 'http://www.wikidata.org/entity/Q178628'), ('name', 'Paolo Rossi')]
[('p', 'http://www.wikidata.org/entity/Q216140'), ('name', 'Stanley Matthews')]
[('p', 'http://www.wikidata.org/entity/Q310657'), ('name', 'Matías Fernandez')]
[('p', 'http://www.wikidata.org/entity/Q483145'), ('name', 'Luís Figo')]
[('p', 'http://www.wikidata.org/entity/Q483577'), ('name', 'Rivaldo')]
[('p', 'http://www.wikidata.org/entity/Q483837'), ('name', 'Luka Modrić')]
[('p', 'http://www.wikidata.org/entity/Q39444'), ('name', 'Ronaldinho')]
[('p', 'http://www.wikidata.org/entity/Q483629'), ('name', 'Marco van Basten')]
[('p', 'http://www.wikidata.org/entity/Q17163'), ('name', 'Johan Cruyff')]
[('p',

## Task 6
Return the countries which have at least two different players of their nationality (consider the country of citizenship) who won the ballon d'Or (the result set must be a list of triples country IRI, label and #different players).

In [36]:
# write your queries

In [167]:
# look at ronaldo's properties for country of citizenship

# wd:Q11571 = Cristiano Ronaldo

queryString = """
SELECT DISTINCT *
WHERE
{
    wd:Q11571 ?p ?obj .
    ?p sc:name ?name .
    
    FILTER REGEX(?name, ".*country.*of.*citizenship.*") .
    
    
}
#GROUP BY ?nationality
#HAVING (COUNT(?nationality) > 2)
#ORDER BY DESC (?awardCount)
"""

print("Results")
x = run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('obj', 'http://www.wikidata.org/entity/Q45'), ('name', 'country of citizenship')]
1


In [175]:
# we know that
# P27 = 'country of citizenship'

# wdt:P166 = award received  
# wd:Q166177 = Ballon d'Or 

queryString = """
SELECT DISTINCT  ?country ?countryName (count(distinct ?player) as ?numberOfPlayers)
WHERE
{
    ?player wdt:P166 wd:Q166177 .
    ?player wdt:P27 ?country .
    
    ?country sc:name ?countryName .
}
group by ?country ?countryName
having (count(distinct ?player) >= 2)
order by desc(?numberOfPlayers)
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('numberOfPlayers', '6')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('numberOfPlayers', '6')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('numberOfPlayers', '5')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('numberOfPlayers', '4')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('numberOfPlayers', '4')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil'), ('numberOfPlayers', '4')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal'), ('numberOfPlayers', '3')]
[('country', 'http://www.wikidata.org/entity/Q212'), ('countryName', 'Ukraine'), ('numberOfPlayers', '3')]
[('country', 'http://www.wikidata.org/entity/Q15180'), ('countryName', 'Soviet Union'), ('numberOfPlayers', '3')]
[('country', 'http://ww

Final query for this task

In [176]:
# write your final query

In [177]:
# we know that
# P27 = 'country of citizenship'

# wdt:P166 = award received  
# wd:Q166177 = Ballon d'Or 

queryString = """
SELECT DISTINCT  ?country ?countryName (count(distinct ?player) as ?numberOfPlayers)
WHERE
{
    ?player wdt:P166 wd:Q166177 .
    ?player wdt:P27 ?country .
    
    ?country sc:name ?countryName .
}
group by ?country ?countryName
having (count(distinct ?player) >= 2)
order by desc(?numberOfPlayers)
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('numberOfPlayers', '6')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('numberOfPlayers', '6')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('numberOfPlayers', '5')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('numberOfPlayers', '4')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('numberOfPlayers', '4')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil'), ('numberOfPlayers', '4')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal'), ('numberOfPlayers', '3')]
[('country', 'http://www.wikidata.org/entity/Q212'), ('countryName', 'Ukraine'), ('numberOfPlayers', '3')]
[('country', 'http://www.wikidata.org/entity/Q15180'), ('countryName', 'Soviet Union'), ('numberOfPlayers', '3')]
[('country', 'http://ww

## Task 7
Identify the BGP of the FIFA World Cup

In [39]:
# write your queries

In [114]:
# is ronaldo related to fifa world cup in any way

# wd:Q11571 = Cristiano Ronaldo

queryString = """
SELECT DISTINCT ?pr ?obj ?prName ?objName
WHERE
{
    wd:Q11571 ?pr ?obj .
    
    ?pr sc:name ?prName .
    ?obj sc:name ?objName .
    
    filter regex(?objName, ".*FIFA.*World.*Cup*") .
}
order by ?objName
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1344'), ('obj', 'http://www.wikidata.org/entity/Q37285'), ('prName', 'participant in'), ('objName', '2006 FIFA World Cup')]
[('pr', 'http://www.wikidata.org/prop/direct/P1344'), ('obj', 'http://www.wikidata.org/entity/Q176883'), ('prName', 'participant in'), ('objName', '2010 FIFA World Cup')]
[('pr', 'http://www.wikidata.org/prop/direct/P1344'), ('obj', 'http://www.wikidata.org/entity/Q79859'), ('prName', 'participant in'), ('objName', '2014 FIFA World Cup')]
[('pr', 'http://www.wikidata.org/prop/direct/P1344'), ('obj', 'http://www.wikidata.org/entity/Q170645'), ('prName', 'participant in'), ('objName', '2018 FIFA World Cup')]
4


In [115]:
# Is 2018 FIFA World Cup related to FIFA worl cup in any way??
# is ronaldo related to fifa world cup in any way

# wd:Q170645 = 2018 FIFA World Cup

queryString = """
SELECT DISTINCT ?pr ?obj ?prName ?objName
WHERE
{
    wd:Q170645 ?pr ?obj .
    
    ?pr sc:name ?prName .
    ?obj sc:name ?objName .
    
    filter regex(?objName, "^FIFA.*World.*Cup.*") .
}
order by ?objName
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P3450'), ('obj', 'http://www.wikidata.org/entity/Q19317'), ('prName', 'sports season of league or competition'), ('objName', 'FIFA World Cup')]
1


In [116]:
# we know that
# Q19317 = FIFA World Cup


queryString = """
SELECT DISTINCT  ?p ?name
WHERE
{
    wd:Q19317 ?p ?obj.
    ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P7441'), ('name', 'WorldFootball.net competition ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1705'), ('name', 'native label')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('name', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct

Final query for this task


In [117]:
# write your final query

In [118]:
# we know that
# Q19317 = FIFA World Cup


queryString = """
SELECT DISTINCT  ?p ?name
WHERE
{
    wd:Q19317 ?p ?obj.
    ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P7441'), ('name', 'WorldFootball.net competition ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1705'), ('name', 'native label')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('name', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct

## Task 8
Who are the players who partecipated in more than 4 world's cup editions? (the result set must be a list of triples player IRI, label and #participations).

In [43]:
# write your queries

In [143]:
# 'sports season of league or competition' property tells us if it is a FIFA World Cup
# need to limit to humans

# wd:Q5 = Human
# P31 = instnace of
# Q19317 = FIFA World Cup
# P3450 = sports season of league or competition

# P1344 = participated in


queryString = """
SELECT DISTINCT  ?player ?playerName (count(?player) as ?participations)
WHERE
{
    ?player wdt:P31 wd:Q5 .
    ?player wdt:P1344 ?fifaWC .
    ?fifaWC wdt:P3450 wd:Q19317 .
    
    ?player sc:name ?playerName .
    ?fifaWC sc:name ?fifaWCName .
}
group by ?player ?playerName
having (count(?player) > 4)
order by desc (?participations)
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q36386'), ('playerName', 'Lothar Matthäus'), ('participations', '5')]
[('player', 'http://www.wikidata.org/entity/Q186330'), ('playerName', 'Rafael Márquez'), ('participations', '5')]
[('player', 'http://www.wikidata.org/entity/Q265428'), ('playerName', 'Antonio Carbajal'), ('participations', '5')]
[('player', 'http://www.wikidata.org/entity/Q68060'), ('playerName', 'Gianluigi Buffon'), ('participations', '5')]
4


Final query for this task

In [49]:
# write your final query

In [144]:
# 'sports season of league or competition' property tells us if it is a FIFA World Cup
# need to limit to humans

# wd:Q5 = Human
# P31 = instnace of
# Q19317 = FIFA World Cup
# P3450 = sports season of league or competition

# P1344 = participated in


queryString = """
SELECT DISTINCT  ?player ?playerName (count(?player) as ?participations)
WHERE
{
    ?player wdt:P31 wd:Q5 .
    ?player wdt:P1344 ?fifaWC .
    ?fifaWC wdt:P3450 wd:Q19317 .
    
    ?player sc:name ?playerName .
    ?fifaWC sc:name ?fifaWCName .
}
group by ?player ?playerName
having (count(?player) > 4)
order by desc (?participations)
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q36386'), ('playerName', 'Lothar Matthäus'), ('participations', '5')]
[('player', 'http://www.wikidata.org/entity/Q186330'), ('playerName', 'Rafael Márquez'), ('participations', '5')]
[('player', 'http://www.wikidata.org/entity/Q265428'), ('playerName', 'Antonio Carbajal'), ('participations', '5')]
[('player', 'http://www.wikidata.org/entity/Q68060'), ('playerName', 'Gianluigi Buffon'), ('participations', '5')]
4


## Task 9
Consider the countries that won the FIFA World Cup at least twice. Find the players of those countries won the ballon d'Or (the result set must be a list of couples player IRI and label).

In [50]:
# write your queries

In [157]:
# checking if FIFA world cup has somthing showing country that won


# P31 = instance of
# Q19317 = FIFA World Cup
# P3450 = sports season of league or competition

# P1344 = participated in


queryString = """
SELECT DISTINCT  ?p ?obj ?pName ?objName
WHERE
{
    
    ?fifaWC wdt:P3450 wd:Q19317 .
    ?fifaWC ?p ?obj .
    
    ?p sc:name ?pName .
    ?obj sc:name ?objName .
    
    FILTER REGEX(?objName, ".*Italy.*") .
}
order by desc (?participations)
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('obj', 'http://www.wikidata.org/entity/Q676899'), ('pName', 'winner'), ('objName', 'Italy national association football team')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('pName', 'country'), ('objName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P1923'), ('obj', 'http://www.wikidata.org/entity/Q676899'), ('pName', 'participating team'), ('objName', 'Italy national association football team')]
3


In [160]:
# find property that links 'Italy national association football team' to Italy
# checking if FIFA world cup has somthing showing country that won

# Q676899 = Italy national association football team


queryString = """
SELECT DISTINCT  ?p ?obj ?pName ?objName
WHERE
{
    
    wd:Q676899 ?p ?obj .
    
    ?p sc:name ?pName .
    ?obj sc:name ?objName .
    
    FILTER REGEX(?objName, ".*Italy.*") .
}
#order by desc (?participations)
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('obj', 'http://www.wikidata.org/entity/Q516025'), ('pName', 'participant in'), ('objName', 'Italy 4–3 West Germany')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('pName', 'country for sport'), ('objName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('pName', 'country'), ('objName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('obj', 'http://www.wikidata.org/entity/Q1114877'), ('pName', 'history of topic'), ('objName', 'history of the Italy national football team')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('obj', 'http://www.wikidata.org/entity/Q7190317'), ('pName', "topic's main category"), ('objName', 'Category:Italy national football team')]
5


In [186]:
# property 'winner' tells us which team won
# property 'country' tells us whcih country that team belongs to

# P31 = instance of
# Q19317 = FIFA World Cup
# P3450 = sports season of league or competition

# P1346 = winner 
# P17 = country

# P27 = 'country of citizenship'
# wdt:P166 = award received  
# wd:Q166177 = Ballon d'Or 

queryString = """
SELECT DISTINCT  ?player ?playerName
{
    {
        SELECT DISTINCT  ?country
        WHERE
        {
            ?fifaWC wdt:P3450 wd:Q19317 .
            ?fifaWC wdt:P1346 ?winner .
            ?winner wdt:P17 ?country .
        }
        group by ?country
        having (count(?winner) >= 2)
    }
    
    ?player wdt:P27 ?country .
    ?player wdt:P166 wd:Q166177 .
    
    ?player sc:name ?playerName
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q4261'), ('playerName', 'Michel Platini')]
[('player', 'http://www.wikidata.org/entity/Q13494'), ('playerName', 'Jean-Pierre Papin')]
[('player', 'http://www.wikidata.org/entity/Q1835'), ('playerName', 'Zinedine Zidane')]
[('player', 'http://www.wikidata.org/entity/Q123118'), ('playerName', 'Raymond Kopa')]
[('player', 'http://www.wikidata.org/entity/Q483577'), ('playerName', 'Rivaldo')]
[('player', 'http://www.wikidata.org/entity/Q39444'), ('playerName', 'Ronaldinho')]
[('player', 'http://www.wikidata.org/entity/Q531814'), ('playerName', 'Kaká')]
[('player', 'http://www.wikidata.org/entity/Q529207'), ('playerName', 'Ronaldo')]
[('player', 'http://www.wikidata.org/entity/Q36386'), ('playerName', 'Lothar Matthäus')]
[('player', 'http://www.wikidata.org/entity/Q43744'), ('playerName', 'Gerd Müller')]
[('player', 'http://www.wikidata.org/entity/Q152871'), ('playerName', 'Karl-Heinz Rummenigge')]
[('player', 'http://www.wikidata.org/entit

Final query for this task

In [51]:
# write your final query